# Experiment: _Remove unseen nodes_

Here, we remove nodes that have not been seen in training.

In [ ]:
from notebook_prelude import *

In [ ]:
df = results_helper.get_experiments_by_names(['experiment_remove_unseen_nodes'], fetch_predictions=True)
print('# Results: {}'.format(len(df)))

In [ ]:
d = df.groupby(['dataset', 'graph_preprocessing']).mean_test_f1_macro.max().to_frame().unstack()
d.columns = d.columns.droplevel()
d = d.rename(columns={'-': 'plain'})
d['difference'] = d['plain'] - d['RemoveNewGraphNodes']
d

In [ ]:
for idx, df_ in df.loc[df[df.dataset=='ng20'].groupby('graph_preprocessing').mean_test_f1_macro.idxmax()].iterrows():
    print(df_.graph_preprocessing)
    res = get_pickle(df_.prediction_file)['results']
    X_test = res['X_test']


In [ ]:
bdf.columns

## _ng20_ test

In [ ]:
X, Y = dataset_helper.get_concept_map_for_dataset('rotten_imdb')
X = graph_helper.get_graphs_only(X)

In [ ]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, stratify=Y, test_size=0.2)
t = transformers.RemoveNewGraphNodes()
print('Fit')
t.fit(X_train)
print('Transform')
print('\tTrain')
X_train_ = t.transform(X_train)
print('\tTest')
X_test_ = t.transform(X_test)

In [ ]:
train_labels = graph_helper.get_all_node_labels_uniq(X_train_, as_sorted_list=False)
test_labels = graph_helper.get_all_node_labels_uniq(X_test_, as_sorted_list=False)
assert len(train_labels) == len(t.train_labels_)
assert len(test_labels) <= len(train_labels)

In [ ]:
labels_c_train = collections.Counter(train_labels)
labels_c_test = collections.Counter(test_labels)
sum(labels_c_test.values())

In [ ]:
t_wl = transformers.FastWLGraphKernelTransformer()

phis_train = t_wl.fit_transform(X_train_)
phis_test = t_wl.transform(X_test_)

In [ ]:
phis_train[0].sum(), phis_test[0].sum()

In [ ]:
np.where(np.array([1, 2]) < 1, 0, 1)

In [ ]:
def get_graphs(as_adj=False):
    g_train = nx.DiGraph()
    g_train.add_edge('A', 'B')
    g_train.add_edge('A', 'C')
    g_train.add_edge('B', 'C')

    g_test = nx.DiGraph()
    g_test.add_edge('A', 'B')
    g_test.add_edge('A', 'C')

    if as_adj:
        g_test = graph_helper.convert_graphs_to_adjs_tuples([g_test], copy=True)[0]
        g_train = graph_helper.convert_graphs_to_adjs_tuples([g_train], copy=True)[0]

    return g_train, g_test


def get_phi_list(X_train = None, X_test = None, **fast_wl_params):
    t = transformers.FastWLGraphKernelTransformer(**fast_wl_params)
    if X_train is None or X_test is None:
        g_train, g_test = get_graphs()
    phi_list_train = t.fit_transform([g_train])
    phi_list_test = t.transform([g_test])
    return phi_list_train, phi_list_test

phis_train, phis_test = get_phi_list(h=5, use_early_stopping=True)
assert len(phis_train) == len(phis_test)
phis_train[-1].todense(), phis_test[-1].todense()